In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np
from tqdm import tqdm

# Atrial fibrillation (AF) patients on warfarin

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'rb') as f:
    warfarin = pickle.load(f)

In [ ]:
len(warfarin)

In [ ]:
df = pd.DataFrame(list(warfarin.items()), columns=['client_idcode', 'first_mention_warfarin']) # This code converts a dictionary (warfarin) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_warfarin'
df.shape

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# Platelet count

In [ ]:
pc = pd.read_csv('/projects/ROCKET AF/data/platelet_count.csv')

In [ ]:
pc.head()

In [ ]:
pc['basicobs_itemname_analysed'].unique()

In [ ]:
pc['basicobs_unitofmeasure'].unique()

In [ ]:
pc.loc[pc['basicobs_itemname_analysed'] == 'Platelet Count (Citrate Sample)']

In [ ]:
pc = pc[['client_idcode', 'basicobs_itemname_analysed', 'basicobs_value_numeric', 'basicobs_unitofmeasure', 'basicobs_value_analysed', 'basicobs_createdwhen']]

In [ ]:
pc.head()

In [ ]:
pc.shape

In [ ]:
pc['first_warfarin_prescription'] = pc['client_idcode'].map(warfarin)

In [ ]:
pc.head()

In [ ]:
pc['basicobs_createdwhen'] = pd.to_datetime(pc['basicobs_createdwhen'], utc=True).dt.date 

In [ ]:
pc.head()

In [ ]:
pc['difference'] = pd.to_datetime(pc['first_warfarin_prescription']) - pd.to_datetime(pc['basicobs_createdwhen'])

In [ ]:
pc.loc[pc['difference'].notna()]

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = pc['difference'] >= min_time 
t2 = pc['difference'] <= max_time
in_window = t1 & t2 
pc['in_window'] = in_window

In [ ]:
pc = pc.loc[pc['in_window'] == True]

In [ ]:
pc.head()

In [ ]:
pc.shape

In [ ]:
# Group by 'client_idcode' and keep only the rows with the smallest 'difference'

min_values = pc.groupby('client_idcode')['difference'].transform('min')

# Filter the DataFrame to keep only rows with the minimum 'Value'
pc = pc[pc['difference'] == min_values]

In [ ]:
pc.head()

In [ ]:
pc.shape

In [ ]:
pc['client_idcode'].nunique()

In [ ]:
pc = pc.sort_values(by='client_idcode')
pc.head()

In [ ]:
# Group by 'client_idcode' and calculate the average of 'platelet count'

df = pc.groupby('client_idcode')['basicobs_value_numeric'].mean().reset_index()
    

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['client_idcode'].nunique()

In [ ]:
df['basicobs_value_numeric'].min()

In [ ]:
df['basicobs_value_numeric'].max()

In [ ]:
df.loc[df['basicobs_value_numeric'] >= 450]

In [ ]:
df.loc[df['basicobs_value_numeric'] >= 800]

In [ ]:
pc.loc[pc['basicobs_value_numeric'] >= 800]

In [ ]:
platelet_count = dict(zip(df['client_idcode'], df['basicobs_value_numeric']))
len(platelet_count)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/platelet_count.pickle', 'wb') as f:
    pickle.dump(platelet_count,f)